In [6]:
import sys
import os

In [17]:
# Add the directory containing oceandirect to the Python path
oceandirect_path = 'C:/Program Files/Ocean Optics/OceanDirect SDK/Python/'  # Replace with the actual path
if os.path.exists(oceandirect_path):
    sys.path.append(os.path.abspath(oceandirect_path))
    print(f"Path exists and added to sys.path: {oceandirect_path}")
else:
    print(f"Path does not exist: {oceandirect_path}")

Path exists and added to sys.path: C:/Program Files/Ocean Optics/OceanDirect SDK/Python/


In [18]:
try:
    from oceandirect.OceanDirectAPI import OceanDirectAPI, OceanDirectError, Spectrometer
    print("Import successful")
except ImportError as e:
    print(f"Import failed: {e}")

Import successful
